# 분류 ANN을 위한 인공지능 모델 구현

In [1]:
! python -V

Python 3.6.8 :: Anaconda, Inc.


In [ ]:
! conda install -y tensorflow

## 1단계: 모듈 import

In [ ]:
from keras import layers, models

첫 번째로 불러온 layers는 각 계층을 만드는 모듈이다. 두 번째로 불러온 models는 각 layer들을 연결하여 신경망 모델을 만든 후, 컴파일하고, 학습시키는 역할을 한다. 물론 학습 후 평가도 models를 이용하여 진행한다. <br>
객체지향 방식을 지원하는 케라스는 models.Model 객체에서 compile(), fit(), predict(), evaluate() 등 딥러닝 처리 함수 대부분을 제공해 편리하게 사용할 수 있다.

## 2단계: 분류 ANN에 필요한 파라미터 설정

분류 ANN에 필요한 파라미터는 하단의 표를 참조한다. 이 값의 실제 정의는 main() 함수 안에서 진행하며 2.2.4 '분류 ANN 학습 및 성능 분석'에서 다시 언급한다. 꼭 전역 변수로 지정할 필요가 없다면 파라미터들을 ㅣ작 함수인 main()에 넣어준다.

| 파라미터 | 의미 |
|:--------|:--------:|
| Nin | 입력계층의 노드 수 | 
| Nh | 은닉계층의 노드 수 |
| number_of_class | 출력값이 가질 클래스 수|
| Nout | 출력 노드 수 |

## 3단계: 모델링 

### 연쇄방식 vs 분산방식
케라스는 인공지능 모델을 연쇄 방식과 분산 방식으로 구현할 수 있다. 연쇄 방식은 간단히 구현할 수 있지만 복잡도가 높은 모델에 적용하기에는 한계가 있다. 따라서 두 방식을 모두 알아두는 것이 좋다. 또한 모델을 구현하는 방식도 함수형과 객체지향형 방법을 모두 다룬다. 이 책에서 고려하는 모델 구현 방식은 다음고 같다.
* 분산 방식 모델링을 포함하는 함수형 구현
* 연쇄 방식 모델링을 포함하는 함수형 구현
* 분산 방식 모델링을 포함하는 객체지향형 구현
* 연쇄 방식 모델링을 포함하는 객체지향형 구현

### 분산 방식, 함수형 구현
ANN 모델을 분산 방식으로 구현한다. 모델 구현에는 함수형 방식을 사용한다. (ANN -> 분산 -> 함수형)

In [2]:
# 분산 방식 모델링을 포함하는 함수형 구현
def ANN_models_func(Nin, Nh, Nout):
    x = layers.Input(shape=(Nin,)) # 입력층
    h = layers.Activation('relu')(layers.Dense(Nh)(x)) #은닉층, x가 입력, 함수는 relu
    y = layers.Activateion('sotfmax')(layers.Dense(Nout)(h)) #출력층, h가 입력, 함수는 softmax
    model = models.Model(x,y)
    model.compile(losss='categorical_crossentropy',
                  optimizer='adam', 
                  metrics=['accuracy'])
    return model

1. 우선, 신경망 구조를 지정해야한다. 이는 입력 계층 정의하는걸로 시작한다. 입력 계층은 layers.Input() 함수로 지정한다. 원소를 Nin개 가지는 입력 신호 벡터는 입력 노드에 따른 계층의 shape을 (Nin.)으로 지정한다.
1. 이어서 은닉 계층의 구조와 수를 정해야 한다. 은닉 계층은 layers.Dense()로 지정한다. 노드가 Nh개인 경우에 은닉 계층을 layers.Dense(Nh)로 구성한다. 이 은닉 계층의 입력은 입력 노드이다. 즉, x를 입력으로 받아들이도록 layers.Dense(Nh)(x)로 지정한다.<br>그리고 활성함수를 layers.Activation('relu')로 지정한다. 여기서 ReLU는 최근에 많이 사용되는 활성화 함수로 f(x) = max(x,0)과 같다. 단순하지만, 딥러닝 학습에 도움이 되기 때문에 최근에는 tanh()나 sigmoid()함수보다 많이 사용된다. 따라서 입력 벡터인 x를 완전히 연결된 은닉 계층의 노드들로 모두 보내고 은닉 계층의 각 노드들은 ReLU로 활성화 처리한 뒤에 다음 계층으로 내보낸다.
1. 마지막 출력 계층, 다중 클래스 분류를 ANN으로 구현하고 있으므로 출력 노드 수는 클래스 수(Nout)로 지정한다. 이때 출력 노드에 입력되는 정보는 은닉 노드의 출력값이다. 또한 분류의 경우에 출력 노드의 활성화 함수로 소프트맥스 연산을 사용한다.
1. 이제 앞에 나온 계층들을 합쳐 인공지능 모델을 만든다. 모델은 입력과 출력을 지정하여 만든다. 중간 계층들은 앞서 정의한 계층 간 신호의 연결 관계대로 작동 설정된다. 여기서 Model은 딥러닝 구조가 여러 가지 딥러닝에 필요한 함수와 연꼐되도록 만드는 역할을 한다.
1. 마지막으로 컴파일을 한다. 케라스는 컴파일을 수행하여 타깃 플랫폼에 맞게 딥러닝 코드를 구성한다. 사실 파이썬은 스크림트 언어이기 때문에 컴파일이 없이도 실행된다. 그렇지만 케라스의 내부 딥러닝 엔진이 시애노이면 CPU인지, GPU인지에 따라 딥러닝 모델과 동작에 대한 부분을 자동으로 컴파일해 수행 속도를 최적화한다.<br> GPU를 사용한다고 지정하면 엔비디아가 제공하는 쿠다(CUDA) 컴파일러를 이용해 실행 코드를 만든다. 만약, 텐서플로를 사용하면 코드를 컴파일하지는 않지만 다양한 초기화 작업을 한다.

note: 컴파일 할때 loss는 손실함수를 지정하는 argument이다. 케라스가 제공하는 손실 함수 외에도 직접 새로운 손실 함수를 제작하여 지정할 수 있다. 두 번째 아규먼트인 optimizer는 최적화 함수를 지정한다. 마지막 argument인 metric은 학습이나 예측이 진행될 때 성증 검증을 위해 손실뿐 아니라 정확도(accuracy)도 측정하라는 의미이다.

#### cf) 객체를 함수처럼 사용
파이썬은 객체를 만든 후에 함수처럼 사용할 수 있다. def __call__(self, )라는 멤버 함수를 사용하면 된다. 예를 들어 Dense라는 객체를 만들어 함수처럼 사용하려면 다음과 같이 하면 된다.
```
class Dense:
    def __call__(self, x):
        print(x)
```
함수호출: Dense()(1)

### 연쇄 방식, 함수형 구현
연쇄 방식은 앞서 살펴본 분산 방식과 모델을 지정하는 부분만 다르고, 상수를 정의하고 설정한 모델을 컴파일하는 구현 코드는 같다.

In [ ]:
# 연쇄 방식 모델링을 포함하는 함수형 구현
def ANN_seq_func(Nin, Nh, Nout):
    model = models.Sequentail()
    model.add(layers.Dnese(Nh, activation='relu', input_shape=(Nin,)))
    model.add(layers.Dense(Nout, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])
    return model

1. 연쇄 방식은 모델 구조를 정의하기 전에 Sequential() 함수로 모델을 초기화해야 한다.
1. 첫 번째 add() 단계에서 입력 계층과 은닉 계층의 형태가 동시에 정해진다. 입력 노드 Nin개는 완전 연결 계층 Nh개로 구성된 은닉 계층으로 보내진다. 이 은닉 계층의 노드들은 ReLU를 활서오하 함수로 사용한다. 
1. 또한 은닉 계층의 출력은 출력이 Nout개인 출력 노드로 보내진다. 출력 노드들의 활성화 함수는 소프트맥스 연산으로 지정했다.
이처럼 연쇄 방식은 추가되는 계층을 기술할 때 간편하게 기술할 수 있다는 장점이 있다. add()를 이용해 연속되는 계층을 계속 더해주면 된다. 다만, 복잡한 인공신경망을 기술하는 부분은 연쇄형 모델링만으로 구현이 힘든 경우도 있다. 이런 경우 분산방식 모델링을 사용해야 한다.

### 분산 방식, 객체지향형 구현
구현할 ANN 코드의 재사용성을 높이기 위해 객체지향 방식으로 구현할 수도 있다. 또한 이 방식은 일반 사용자의 경우 전문가가 만든 인공지능 모델을 객체로 불러 쉽게 활용할 수 있다는 장점이 있다.<br>
객체지향 방식 구현을 위해서는 먼저 클래스를 만들고, models.Model로부터 특성을 상속해온다. model.Model은 신경망에서 사용하는 학습, 예측, 평가와 같은 다양한 함수를 제공한다.

In [ ]:
# 분산 방식 모델링을 포함하는 객체지향형 구현
class ANN_models_class(models.Model):
    def __init__(self, Nin, Nh, Nout):
        # Prepare network layers and activate functions / 모델에 사용할 계층 정의
        hidden = layers.Dense(Nh) 
        output = layers.Dense(Nout)
        relu = layers.Activation('relu')
        softmax = layers.Activation('softmax')
        
        # Connect network elements
        x = layers.Input(shape=(Nin,))
        h = relu(hidden(x))
        y = softmax(output(h))
        
        super().__init__(x,y)
        self.compile(loss='categorical_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])

1. 클래스의 초기화 함수를 정의한다. 초기화 함수는 입력 계층, 은닉 계층, 출력 계층의 노드 수를 각각 Nin, Nh, Nout으로 받는다. 만약 모델에 대한 함수를 추가로 구현하지 않고 model에서 기본 제공하는 함수만 이용한다면 초기화 함수인 __init__()만 정의해 클래스를 구현할 수 있다. 각 계층의 노드 수를 클래스의 초깃값으로 사용하므로 나중에 ANN 인스턴스를 만들 때 이 값들을 유연하게 설정할 수 있다.
1. 이후 신경망의 모델에 사용할 계층를 정의한다. 이번 ANN에는 은닉 계층이 하나이므로 은닉 계층의 출력 변수로 hidden 하나만 사용했다.

cf) 은닉 계층이 다수일 경우
```
hidden_l = []
for n in Nh_l
    hidden_l.append(layers.Dense(n))
```
은닉 계층이 여럿일때는 반복문을 사용해서 생성해야 한다. 이때는 각 계층마다 노드 수가 Nh_l = [5, 10, 5]라고 한다. <br>
은닉 계층을 hidden_l = map(layers.Dense, Nh_l)과 같이 반복문을 사용해서 만들 수 있다. <br>
for문을 사용해 hidden_l=[layers.Dense(n) for n in Nh_l]과 같이 표현할 수 있다.